In [2]:
from models.Yet_Another_YOLOv4_Pytorch.model import YOLOv4
from models.Yet_Another_YOLOv4_Pytorch.pl_model import YOLOv4PL
from models.Yet_Another_YOLOv4_Pytorch.dataset import ListDataset_Test
from models.Yet_Another_YOLOv4_Pytorch.utils import get_bboxes_from_anchors, get_img_with_bboxes,get_img_with_bboxes_gt
import numpy as np
import cv2
import torch
from torch.utils.data import DataLoader
#import utils
import time
from pytorch_lightning import Trainer
from torchvision.utils import save_image
from PIL import Image
import tqdm
import os


In [ ]:
#model = torch.load('lightning_logs/final/fire/2t12n2tq_epoch=73-step=40848.ckpt')

# model['hyper_parameters']['augmentation_on'] = True
# model['hyper_parameters']['train_part'] = ['backbone','neck','head']
# model['hyper_parameters']['pretrained'] = False
# model['hyper_parameters']['weights_path'] = False
# torch.save(model, 'lightning_logs/final/fire/2t12n2tq_epoch=73-step=40848_replace_hparams.ckpt')

In [3]:
model_pl = YOLOv4PL.load_from_checkpoint('lightning_logs/final/fog/epoch=18-step=9804.ckpt',pretrained = False,weights_path = False)
model_pl.eval()


Dataset size:4055
Dataset size:1158
Dataset size:581


YOLOv4PL(
  (model): YOLOv4(
    (backbone): Backbone(
      (d1): DownSampleFirst(
        (c1): ConvBlock(
          (module): Sequential(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Mish()
          )
        )
        (c2): ConvBlock(
          (module): Sequential(
            (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Mish()
          )
        )
        (c3): ConvBlock(
          (module): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Mish()
          )
        )
        (c4): ConvBlock(
          (module): Seq

In [16]:
import models.Yet_Another_YOLOv4_Pytorch.utils as utils
dataset = ListDataset_Test('datasets/fire_and_smoke_detect/detection/Fog/test/images', img_extensions=['.jpg'])
dataloader = DataLoader(dataset)
if not os.path.exists('fog_predictions'):
    os.makedirs('fog_predictions')
    os.makedirs('fog_predictions/images')
    os.makedirs('fog_predictions/labels')
with tqdm.tqdm(total=len(dataloader)) as pbar:
    for batch in dataloader:
        image_path, image = batch
        _,_,h,w = image.shape
        # get original h,w
        with Image.open(image_path[0]) as img:
            w_original,h_original = img.size
        if w_original > h_original:
            ratio = h_original/w_original
            t_width = 608
            t_height = int(ratio * 608)
        else:
            ratio = w_original/h_original
            t_width = int(ratio * 608)
            t_height = 608

        image = image.to(model_pl.device)
        anchors, _ = model_pl(image)
        iou_threshold = 0.01#0.5
        confidence_threshold = 0.5#0.4
        bboxes, pred_labels = get_bboxes_from_anchors(anchors, 
                                                        confidence_threshold, 
                                                        iou_threshold,
                                                        {0:'object'})
        gt_label_path = image_path[0].replace('images','labels').replace('.jpg','.txt')
        
        
        # with open(gt_label_path, 'r') as f:
        #     gt_labels = f.readlines()
        #     # to tensor
            
        #     gt_labels = torch.tensor([list(map(float, label.split())) for label in gt_labels])
        #     gt_labels = gt_labels[:,1:]
        #     gt_labels[:,0] = gt_labels[:,0]*w
        #     gt_labels[:,1] = gt_labels[:,1]*h
        #     gt_labels[:,2] = gt_labels[:,2]*w
        #     gt_labels[:,3] = gt_labels[:,3]*h
        #     #gt_labels[:,0] -= gt_labels[:,2]/2
        #     #gt_labels[:,1] -= gt_labels[:,3]/2
        #     # reshape Nx4->Nx6
        gt_boxes = torch.from_numpy(np.loadtxt(gt_label_path).reshape(-1, 5))
        xyxy_bboxes = utils.xywh2xyxy(gt_boxes[:, 1:])

            #IMG
        padding = abs((t_width - t_height))//2
        relative_padding = padding/w
        #BOXES
        if t_width > t_height:
            #Change y's relative position
            xyxy_bboxes[:, 1] *= ratio
            xyxy_bboxes[:, 3] *= ratio
            xyxy_bboxes[:, 1] += relative_padding
            xyxy_bboxes[:, 3] += relative_padding
        else:#x's
            xyxy_bboxes[:, 0] *= ratio
            xyxy_bboxes[:, 2] *= ratio
            xyxy_bboxes[:, 0] += relative_padding
            xyxy_bboxes[:, 2] += relative_padding

        gt_boxes[:, 1:] = utils.xyxy2xywh(xyxy_bboxes)
        #gt_labels = torch.cat((gt_labels,torch.zeros(gt_labels.shape[0],2)+1),dim=1)
        #gt_bboxes = gt_labels

        image = image.squeeze(0)
        bboxes = bboxes[0]
        labels = ['PRED']*255            
        image = get_img_with_bboxes(image.cpu(), bboxes.cpu(), resize=False, labels=labels)

        # cut the center part of original_h,original_w
        image = torch.tensor(image).permute(2,0,1)
        # uint8->float32 255-0.x
        image = image.float()/255
        #image = image[:,int(h/2-result_h/2):int(h/2+result_h/2),int(w/2-result_w/2):int(h/2+result_w/2)]
        labels = ['GT']*255
        gt_boxes[:,1] = gt_boxes[:,1]*w
        gt_boxes[:,2] = gt_boxes[:,2]*h
        gt_boxes[:,3] = gt_boxes[:,3]*w
        gt_boxes[:,4] = gt_boxes[:,4]*h
        gt_boxes = gt_boxes[:,1:]
        #     gt_labels[:,1] = gt_labels[:,1]*h
        #     gt_labels[:,2] = gt_labels[:,2]*w
        #     gt_labels[:,3] = gt_labels[:,3]*h
        image = get_img_with_bboxes_gt(image, gt_boxes, resize=False, labels=labels)
        # numpy permute
        # RGB -> BGR
        save_name = os.path.basename(image_path[0])
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(f'fog_predictions/images/{save_name}', image)
        save_name = save_name.replace('.jpg', '.txt')
        with open(f'fog_predictions/labels/{save_name}', 'w') as f:
            image_size,_,_ = image.shape
            for bbox in bboxes:
                bbox = bbox.cpu().numpy()
                label = 1
                x = bbox[0]/image_size
                y = bbox[1]/image_size
                w = bbox[2]/image_size
                h = bbox[3]/image_size
                f.write(f'{label} {x} {y} {w} {h}\n')
        pbar.update(1)


  0%|          | 0/295 [00:00<?, ?it/s]

100%|██████████| 295/295 [00:20<00:00, 14.15it/s]
